#  Exercise 1: 1) compute maximum length of a codeword with Huffman coding

https://www.coursera.org/learn/algorithms-greedy/exam/u4WGg/programming-assignment-3

In [23]:
import numpy as np

uploaded_vertices = open('./huffman.txt', 'r')
data = uploaded_vertices.read()

weights = [int(x) for x in data.split('\n')[1:-1]]

n_merges = [0] * len(weights)

def find_min_pair(l):
    first_min = np.inf
    second_min = np.inf
    first_min_idx = None
    for i, v in enumerate(l):
        if v < first_min:
            second_min = first_min
            second_min_idx = first_min_idx
            first_min = v
            first_min_idx = i
        elif v < second_min:
            second_min = v
            second_min_idx = i
    return first_min_idx, second_min_idx
        
    
while len(weights) > 1:
    kept_idx, remove_idx = find_min_pair(weights)
    weights[kept_idx] += weights[remove_idx]
    n_merges[kept_idx] += 1
    n_merges[remove_idx] += 1
    weights.pop(remove_idx)

In [24]:
max(n_merges)

16

In [25]:
min(n_merges)

0

In [ ]:
n_merges